In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros para Montecarlo
f = lambda x: (np.cos(x**2))**2
h = lambda x: np.zeros_like(x)
g = lambda x: np.zeros_like(x)
n = int(1e3)
a,b = (-np.pi/2), (np.pi/2)
x = np.linspace(a, b, int(n))

# # Parámetros para muestro por importancia
# f = lambda x: (np.exp(x) + 1)**(-1) * np.sqrt(x)**(-1)
# h = lambda x: 1 + 0 * x
# g = lambda x: np.sqrt(x)**(-1)
# n = int(1e3)
# a,b = (-np.pi/2), (np.pi/2)
# x = np.linspace(a, b, int(n))

# Parámetros para Gauss
A = np.array([[4, 2, 5],
             [2, 5, 8],
             [5, 4, 3]])

B = np.array([[60.7],
             [92.9],
             [56.3]])

# Parámetros para Gauss-Seidel
AS = np.array([[3, -0.1, -0.2],
             [0.1, 7, -0.3],
             [0.3, -0.2, 10]])

BS = np.array([[7.85],
             [-19.3],
             [71.4]])



In [2]:
# Método de Montecarlo sencillo

def MontecarloSimple(funct, iteration, lim):
    sum = 0
    x = np.linspace(a, b, int(iteration))
    f_max = np.max(funct(x))
    y = np.random.uniform(0, f_max, int(iteration))

    # This "for" performos the task of searching for the random values of y that are under the f(x) function.
    for i in range(int(iteration)):
        if(y[i] <= funct(x[i])):
            sum += 1
        
    integral = (lim[1] - lim[0]) * f_max * (sum/iteration)

    return integral
    
_MontecarloSimple = MontecarloSimple(f, n, [a, b])
print(_MontecarloSimple)

1.8723892215280717


In [3]:
# Método de Montecarlo Pro

def MontecarloPro(funct, iteration, lim, weight):
    yRandom = np.random.uniform(0, 1, int(iteration))
    
    if((1/(np.abs(lim[0]) + 1)) < 1e-20) or ((1/(np.abs(lim[0]) + 1)) < 1e-20):
        x = (1/yRandom) - 1
        
        functionValue = (1/yRandom)**2 * funct(x) * weight(x)

    else:
        x = yRandom * (lim[1] - lim[0]) + lim[0]
        
        functionValue = (lim[1] - lim[0]) * funct(x) * weight(x)

    integral = np.mean(functionValue)
    
    return integral
    
_MontecarloPro = MontecarloPro(f, n, [a, b], h)
print(_MontecarloPro)

0.0


In [4]:
# Muestro por importancia
def ImportanceSampling(funct, iteration, weight, indeter):
    yRandom = np.random.uniform(0, 1, int(iteration))**2

    functionValue = funct(yRandom) / g(yRandom)

    integral = np.mean(functionValue)
    
    return integral
    
_ImportanceSampling = ImportanceSampling(f, n, h, g)
print(_ImportanceSampling)

inf


/tmp/ipykernel_125728/1277755130.py:5: RuntimeWarning: divide by zero encountered in divide
  functionValue = funct(yRandom) / g(yRandom)


In [5]:
# Sistemas de Ecuaciones Lineales, Método de Gauss:
def Gauss(matrix, vector):
    systemEq = np.concatenate((matrix,vector), axis=1)
    n, m = systemEq.shape
    
    x = np.zeros(n, dtype = "float")

    # Pivoteo parcial
    for i in range(n):
        column = systemEq[i:,i]
        wheremax = np.argmax(column)
        
        if wheremax != 0:
            temp = np.copy(systemEq[i,:])
            systemEq[i,:] = systemEq[wheremax + i,:]
            systemEq[wheremax + i,:] = temp

    # Eliminación hacia adelante
    for i in range(n):
        pivot = systemEq[i,i]
        forward = i + 1
        
        for j in range(forward, n):
            factor = systemEq[j,i] / pivot
            systemEq[j,:] -= systemEq[i,:] * factor

    # Sustitución hacia atrás
    for i in range(n-1, -1, -1):
        b = systemEq[i, m-1]
        x[i] = (b - np.dot(systemEq[i, :m-1], x)) / systemEq[i,i]
            
    return systemEq, x
            

_Gauss = Gauss(A, B)
print(_Gauss[0])
print("\n")
print(_Gauss[1])
print("\n")
print(A@np.transpose([_Gauss[1]]))

[[ 5.    4.    3.   56.3 ]
 [ 0.    3.4   6.8  70.38]
 [ 0.    0.    5.   40.5 ]]


[2.8 4.5 8.1]


[[60.7]
 [92.9]
 [56.3]]


In [6]:
# Método de Gauss-Jordan:
def GaussJordan(matrix, vector):
    systemEq = np.concatenate((matrix,vector), axis=1)
    n, m = systemEq.shape
    
    x = np.zeros(n, dtype = "float")

    # Pivoteo parcial
    for i in range(n):
        column = systemEq[i:,i]
        wheremax = np.argmax(column)
        
        if wheremax != 0:
            temp = np.copy(systemEq[i,:])
            systemEq[i,:] = systemEq[wheremax + i,:]
            systemEq[wheremax + i,:] = temp

    # Eliminación hacia adelante
    for i in range(n):
        pivot = systemEq[i,i]
        forward = i + 1
        
        for j in range(forward, n):
            factor = systemEq[j,i] / pivot
            systemEq[j,:] -= systemEq[i,:] * factor

    # Eliminación hacia atrás
    for i in range(n - 1, -1, -1):
        pivot = systemEq[i,i]
        downward = i - 1
        
        for j in range(downward, -1, -1):
            factor = systemEq[j,i] / pivot
            systemEq[j,:] -= systemEq[i,:] * factor

    x = systemEq[:, m - 1]*(1/np.diag(systemEq))
            
    return systemEq, x
            

_GaussJordan = GaussJordan(A, B)
print(_GaussJordan[0])
print("\n")
print(_GaussJordan[1])
print("\n")
print(A@np.transpose([_GaussJordan[1]]))

[[ 5.00000000e+00  0.00000000e+00 -1.04491579e-15  1.40000000e+01]
 [ 0.00000000e+00  3.40000000e+00  8.88178420e-16  1.53000000e+01]
 [ 0.00000000e+00  0.00000000e+00  5.00000000e+00  4.05000000e+01]]


[2.8 4.5 8.1]


[[60.7]
 [92.9]
 [56.3]]


In [7]:
# Método de Gauss-Seidel:
def GaussSeidel(matrix, vector, tol, maxIter):
    systemEq = np.concatenate((matrix,vector), axis=1)
    n, m = systemEq.shape
    err = np.inf
    j = 0
    
    x = np.zeros(n, dtype = "float")
    new = np.zeros(n, dtype = "float")

    while not(np.abs(err) < tol or j > maxIter):
        
        for i in range(n):
            temp = np.copy(systemEq)
            tempx = np.copy(x)
            b = systemEq[i, m-1]
            new[i] = (b - np.dot(np.delete(temp[i, :m-1], i), np.delete(tempx, i))) / systemEq[i,i]

        dif = new - x
        x = new
        err = np.max(dif)
        j += 1
        print(j)
                
    return systemEq, x
            
_GaussSeidel = GaussSeidel(AS, BS, 1e-200, 100)
print(_GaussSeidel[0])
print("\n")
print(_GaussSeidel[1])
print("\n")
print(AS@np.transpose([_GaussSeidel[1]]))

1
2
[[  3.    -0.1   -0.2    7.85]
 [  0.1    7.    -0.3  -19.3 ]
 [  0.3   -0.2   10.    71.4 ]]


[ 3.0007619  -2.49401088  7.00009693]


[[  7.85166742]
 [-19.25802908]
 [ 71.4       ]]


In [8]:
# Descomposición LU
def Decomposer(matrix, vector):
    matrix0 = np.array(matrix, dtype = "float")
    n, m = matrix0.shape
    L = np.zeros((n,m), dtype = "float")
    U = np.copy(matrix0)
    x = np.zeros(n, dtype = "float")
    y = np.zeros(n, dtype = "float")
    
    for i in range(n):
        pivot = matrix0[i, i]
        forward = i + 1
        L[i,i] = 1
        
        for j in range(forward, n):
            factor = matrix0[j, i] / pivot
            L[j, i] = factor
            matrix0[j, :] -= matrix0[i, :] * factor
            U[j, :] = np.copy(matrix0[j, :])

    for i in range(n):
        y[i] = (vector[i] - np.dot(L[i,:], y)) / L[i,i]

    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i,:], x)) / U[i,i]

    det = np.prod(np.diag(U))

    return L, U, x, det

_Decomposer= Decomposer(A, B)
print(_Decomposer[0])
print("\n")
print(_Decomposer[1])
print("\n")
print(_Decomposer[0]@_Decomposer[1])
print("\n")
print(A)
print("\n")
print(A@np.transpose([_Decomposer[2]]))
print("\n")
print(_Decomposer[3])

[[1.    0.    0.   ]
 [0.5   1.    0.   ]
 [1.25  0.375 1.   ]]


[[ 4.      2.      5.    ]
 [ 0.      4.      5.5   ]
 [ 0.      0.     -5.3125]]


[[4. 2. 5.]
 [2. 5. 8.]
 [5. 4. 3.]]


[[4 2 5]
 [2 5 8]
 [5 4 3]]


[[60.7]
 [92.9]
 [56.3]]


-85.0


/tmp/ipykernel_125728/2409455318.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = (vector[i] - np.dot(L[i,:], y)) / L[i,i]


In [25]:
# Factorización QR
def Factorizer(matrix, vector):
    matrix0 = np.array(matrix, dtype = "float")
    n, m = matrix0.shape
    Q = np.zeros((n,m), dtype = "float")
    R = np.zeros((m,m), dtype = "float")
    u = np.zeros((n,m), dtype = "float")
    e = np.zeros((n,m), dtype = "float")
    x = np.zeros(n, dtype = "float")
    u[:,0] = matrix[:,0]
    e[:,0] = u[:,0] / np.linalg.norm(u[:,0])

    for i in range(m):
        u[:,i] = matrix0[:,i]
        
        for j in range(i):
            u[:,i] -= ((matrix0[:,i]@e[:,j]) * e[:,j])
            
        e[:,i] = u[:,i] / np.linalg.norm(u[:,i])
        
    for i in range(n):
        for j in range(i, m):
            if i <= j:
                R[i,j] = (matrix0[:,j]@e[:,i])
                          
    Q = np.copy(u)
    for i in range(n - 1, -1, -1):
        x[i] = ((Q.T[i,:]@vector) - np.dot(R[i,:], x)) / R[i,i]

    return Q, R, x

_Factorizer= Factorizer(A, B)
print(_Factorizer[0])
print("\n")
print(_Factorizer[1])
print("\n")
print(_Factorizer[0]@_Factorizer[1])
print("\n")
print(A)
print("\n")
print(A@np.transpose([_Factorizer[2]]))

[[ 4.         -1.37777778  2.48709122]
 [ 2.          3.31111111  0.8777969 ]
 [ 5.         -0.22222222 -2.34079174]]


[[6.70820393 5.66470554 7.60263112]
 [0.         3.59320346 5.26920714]
 [0.         0.         3.52639421]]


[[26.83281573 17.70818629 31.92119207]
 [13.41640786 23.22690699 35.74765047]
 [33.54101966 27.52503806 28.58766627]]


[[4 2 5]
 [2 5 8]
 [5 4 3]]


[[413.38037812]
 [431.64299226]
 [449.33762309]]


/tmp/ipykernel_125728/1031757053.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i] = ((Q.T[i,:]@vector) - np.dot(R[i,:], x)) / R[i,i]
